
### 1. Having Access to `MDLS_API_KEY` from `.env` File

With this approach, you avoid writing an API key in the code. Instead, you put it in the `.env` file. The code below reads the variables in the `.env` file and sets them as environment variables.
In this particular case, we only defined one variable: `MDLS_API_KEY`.

**Important**: put the `.env` file in the same location as this file, or the file that will call the LLM. For example, if the LLM is going to be called from an AHK file, then place the `.env` file in the same location as the AHK file.

The contents of the `.env` file should look like this:
```plaintext
Key_1=Value_1
Key_2=Value_2
...
Key_n=Value_n
```
**To check use:** 
```plaintext
    1. print(os.environ.get("MDLS_API_KEY")), or 
    2. print(os.environ["MDLS_API_KEY"])
```

In [1]:
import os

# Read variables from .env file and sets them as environment variables.
with open(".env") as f:
    for line in f:
        key, value = line.strip().split("=")
        os.environ[key] = value

# Get the environment var MDLS_API_KEY and use it in a print statement to check. 
# You can use it for anything you want, including to configure and use a LLM model :)
# print(f'MDLS_API_KEY is: {os.environ["MDLS_API_KEY"]}')

### 2. Pass MDLS_API_KEY to Generative-AI library and instantiates the model to generate Text from Text (zero-shot approach)

In [ ]:
import google.generativeai as genai 

genai.configure(api_key=os.environ["MDLS_API_KEY"])
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content("Cuanto es la suma de 6 y 6?")
print(response.text)

### 3. Pass MDLS_API_KEY to Generative-AI library and instantiates the model to generate Text from Text & Image (zero-shot approach)

In [ ]:
import google.generativeai as genai #
import PIL.Image

genai.configure(api_key=os.environ["MDLS_API_KEY"]) #
model = genai.GenerativeModel(model_name="gemini-1.5-flash") #
organ = PIL.Image.open("media/organ.jpg")
response = model.generate_content(["Tell me about this instrument", organ])
print(response.text)